## 1. Environment Setup

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
!pip install -q python-chess pandas pillow tqdm opencv-python matplotlib seaborn scikit-learn

## 2. Upload Code and Data

In [ ]:
# Upload code.zip
from google.colab import files
import zipfile
import os
import shutil

print("Upload code.zip...")
uploaded = files.upload()

with zipfile.ZipFile('code.zip', 'r') as z:
    for member in z.namelist():
        z.extract(member, 'temp')

if os.path.exists('temp'):
    for root, dirs, filelist in os.walk('temp'):
        for f in filelist:
            old_path = os.path.join(root, f)
            new_path = os.path.relpath(old_path, 'temp').replace('\\', '/')
            directory = os.path.dirname(new_path)
            if directory:
                os.makedirs(directory, exist_ok=True)
            shutil.copy(old_path, new_path)
    shutil.rmtree('temp')

print("\nCode uploaded!")
!ls src/ dataset_tools/

In [ ]:
# Upload data
import glob

print("Upload all_games_data.zip (5 games: 2,4,5,6,7)...")
uploaded = files.upload()

with zipfile.ZipFile(list(uploaded.keys())[0], 'r') as z:
    for member in z.namelist():
        z.extract(member, 'temp')

if os.path.exists('temp'):
    for root, dirs, filelist in os.walk('temp'):
        for f in filelist:
            old_path = os.path.join(root, f)
            new_path = os.path.relpath(old_path, 'temp').replace('\\', '/')
            directory = os.path.dirname(new_path)
            if directory:
                os.makedirs(directory, exist_ok=True)
            shutil.copy(old_path, new_path)
    shutil.rmtree('temp')

print("\n" + "="*60)
print("GAMES UPLOADED:")
print("="*60)
total_frames = 0
for game in sorted(glob.glob('Data/game*_per_frame')):
    game_name = os.path.basename(game)
    frames = len(glob.glob(f'{game}/tagged_images/*.jpg'))
    total_frames += frames
    print(f"  {game_name}: {frames} frames")
print(f"\nTotal: {total_frames} frames")
print(f"Total games: 5")
print("="*60)

## 3. Prepare Datasets

In [ ]:
import pandas as pd
import numpy as np
import glob
import json
import os
from dataset_tools.fen_utils import PIECE_TO_ID, fen_board_to_64_labels, idx_to_square_name

print("="*60)
print("PREPARING DATASETS")
print("="*60)

os.makedirs('dataset_out', exist_ok=True)

with open('dataset_out/classes.json', 'w') as f:
    json.dump({str(v): k for k, v in PIECE_TO_ID.items()}, f, indent=2)

# Load all games
game_dirs = sorted(glob.glob('Data/*_per_frame'))
game_data = {}

for game_dir in game_dirs:
    game_id = os.path.basename(game_dir)
    csv_file = glob.glob(f'{game_dir}/*.csv')

    if not csv_file:
        continue

    df = pd.read_csv(csv_file[0])
    frame_col = 'from_frame' if 'from_frame' in df.columns else 'frame_id'

    game_rows = []

    for _, r in df.iterrows():
        frame_id = int(r[frame_col])
        fen = r['fen']
        labels = fen_board_to_64_labels(fen)

        frame_path = f'{game_dir}/tagged_images/frame_{frame_id:06d}.jpg'
        if not os.path.exists(frame_path):
            continue

        for sq in range(64):
            game_rows.append({
                'frame_path': frame_path,
                'game_id': game_id,
                'frame_id': frame_id,
                'square_idx': sq,
                'row': sq // 8,
                'col': sq % 8,
                'square_name': idx_to_square_name(sq),
                'label_id': labels[sq],
            })

    game_df = pd.DataFrame(game_rows)
    game_data[game_id] = game_df
    
    n_frames = game_df['frame_id'].nunique()
    n_squares = len(game_df)
    print(f"{game_id}: {n_frames} frames, {n_squares:,} squares")

print(f"\nTotal games loaded: {len(game_data)}")

# Create 2-fold cross-validation splits (for proof of learning)
print("\n" + "="*60)
print("CREATING 2-FOLD CROSS-VALIDATION SPLITS")
print("="*60)

fold_manifests = []
game_list = list(game_data.keys())

# Create 2 folds: each holds out 1 game for testing
for fold_idx in range(2):
    test_game = game_list[fold_idx]
    print(f"\nFold {fold_idx + 1}: Test on {test_game}")
    
    # Combine other 4 games for training
    train_val_dfs = []
    for game_id, game_df in game_data.items():
        if game_id != test_game:
            train_val_dfs.append(game_df.copy())
    
    train_val_df = pd.concat(train_val_dfs, ignore_index=True)
    test_df = game_data[test_game].copy()
    
    # Split training data into 80% train, 20% val
    unique_frames = train_val_df.groupby('game_id')['frame_id'].unique()
    
    train_frames_list = []
    val_frames_list = []
    
    for game_id, frames in unique_frames.items():
        frames = np.array(list(frames))
        n_frames = len(frames)
        
        rng = np.random.RandomState(42)
        rng.shuffle(frames)
        
        n_train = int(0.8 * n_frames)
        train_frames_list.extend([(game_id, f) for f in frames[:n_train]])
        val_frames_list.extend([(game_id, f) for f in frames[n_train:]])
    
    train_frame_set = set(train_frames_list)
    val_frame_set = set(val_frames_list)
    
    def assign_split_train_val(row):
        key = (row['game_id'], row['frame_id'])
        if key in train_frame_set:
            return 'train'
        elif key in val_frame_set:
            return 'val'
        else:
            return None
    
    train_val_df['split'] = train_val_df.apply(assign_split_train_val, axis=1)
    test_df['split'] = 'test'
    
    fold_df = pd.concat([train_val_df, test_df], ignore_index=True)
    fold_df = fold_df.sample(frac=1.0, random_state=42).reset_index(drop=True)
    
    manifest_path = f'dataset_out/fold_{fold_idx + 1}_manifest.csv'
    fold_df.to_csv(manifest_path, index=False)
    
    fold_manifests.append({
        'fold': fold_idx + 1,
        'test_game': test_game,
        'manifest_path': manifest_path,
        'train_squares': (fold_df['split'] == 'train').sum(),
        'val_squares': (fold_df['split'] == 'val').sum(),
        'test_squares': (fold_df['split'] == 'test').sum(),
    })
    
    print(f"  Train: {fold_manifests[-1]['train_squares']:,} squares (4 games)")
    print(f"  Val:   {fold_manifests[-1]['val_squares']:,} squares (20% of 4 games)")
    print(f"  Test:  {fold_manifests[-1]['test_squares']:,} squares (1 game: {test_game})")

# Create combined dataset (all 5 games) for final training
print("\n" + "="*60)
print("CREATING COMBINED DATASET (ALL 5 GAMES)")
print("="*60)

all_games_df = pd.concat([df.copy() for df in game_data.values()], ignore_index=True)

# Split all games into 80% train, 20% val
unique_frames_all = all_games_df.groupby('game_id')['frame_id'].unique()

train_frames_all = []
val_frames_all = []

for game_id, frames in unique_frames_all.items():
    frames = np.array(list(frames))
    n_frames = len(frames)
    
    rng = np.random.RandomState(42)
    rng.shuffle(frames)
    
    n_train = int(0.8 * n_frames)
    train_frames_all.extend([(game_id, f) for f in frames[:n_train]])
    val_frames_all.extend([(game_id, f) for f in frames[n_train:]])

train_frame_set_all = set(train_frames_all)
val_frame_set_all = set(val_frames_all)

def assign_split_all(row):
    key = (row['game_id'], row['frame_id'])
    if key in train_frame_set_all:
        return 'train'
    elif key in val_frame_set_all:
        return 'val'
    else:
        return None

all_games_df['split'] = all_games_df.apply(assign_split_all, axis=1)
all_games_df = all_games_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

all_games_df.to_csv('dataset_out/all_games_manifest.csv', index=False)

train_count = (all_games_df['split'] == 'train').sum()
val_count = (all_games_df['split'] == 'val').sum()

print(f"Train: {train_count:,} squares (80% of all 5 games)")
print(f"Val:   {val_count:,} squares (20% of all 5 games)")
print("\n✓ All datasets prepared!")
print("="*60)

## 4. Quick 2-Fold Validation (Proof of Learning)

Train 2 folds to prove the model learns effectively across different games.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import json
import os
from tqdm import tqdm
import time
import datetime
import random
import numpy as np

# Set seeds
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

print("="*60)
print("2-FOLD CROSS-VALIDATION (PROOF OF LEARNING)")
print("="*60)
print(f"Start Time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Load classes
with open('dataset_out/classes.json', 'r') as f:
    classes = json.load(f)
num_classes = len(classes)

# Dataset class
class ChessSquareDataset(Dataset):
    def __init__(self, manifest_df, transform=None):
        self.data = manifest_df.reset_index(drop=True)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = row['frame_path']
        label = int(row['label_id'])
        
        img = Image.open(img_path).convert('RGB')
        W, H = img.size
        sq_w, sq_h = W // 8, H // 8
        col, row_sq = row['col'], row['row']
        
        left = col * sq_w
        top = row_sq * sq_h
        right = left + sq_w
        bottom = top + sq_h
        
        img_crop = img.crop((left, top, right, bottom))
        
        if self.transform:
            img_crop = self.transform(img_crop)
        
        return img_crop, label

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Training functions
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(dataloader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / total, 100. * correct / total

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Validation", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / total, 100. * correct / total

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Training config
EPOCHS = 8
BATCH_SIZE = 128
LR = 0.001
NUM_WORKERS = 2

# Results storage
fold_results = []

# Train 2 folds
for fold_idx in range(2):
    fold_info = fold_manifests[fold_idx]
    fold_num = fold_info['fold']
    test_game = fold_info['test_game']
    manifest_path = fold_info['manifest_path']
    
    print("\n" + "="*60)
    print(f"FOLD {fold_num}/2 - Testing on {test_game}")
    print("="*60)
    
    # Load data
    manifest_df = pd.read_csv(manifest_path)
    train_df = manifest_df[manifest_df['split'] == 'train']
    val_df = manifest_df[manifest_df['split'] == 'val']
    test_df = manifest_df[manifest_df['split'] == 'test']
    
    print(f"Train: {len(train_df):,} squares")
    print(f"Val:   {len(val_df):,} squares")
    print(f"Test:  {len(test_df):,} squares")
    
    # Create datasets
    train_dataset = ChessSquareDataset(train_df, transform=train_transform)
    val_dataset = ChessSquareDataset(val_df, transform=val_transform)
    test_dataset = ChessSquareDataset(test_df, transform=val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    
    # Initialize model
    model = models.resnet50(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    
    # Training loop
    fold_start = time.time()
    best_val_acc = 0.0
    best_epoch = 0
    
    for epoch in range(EPOCHS):
        epoch_start = time.time()
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"  Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
        print(f"  Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
        print(f"  Time:  {time.time()-epoch_start:.1f}s")
        
        scheduler.step(val_acc)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch + 1
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'fold': fold_num
            }, f'dataset_out/fold_{fold_num}_best.pth')
            print(f"  ✓ Best model saved")
    
    # Test
    checkpoint = torch.load(f'dataset_out/fold_{fold_num}_best.pth', weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    test_loss, test_acc = validate(model, test_loader, criterion, device)
    
    fold_time = time.time() - fold_start
    
    print(f"\nFold {fold_num} Results:")
    print(f"  Best Val Acc:  {best_val_acc:.2f}%")
    print(f"  Test Acc:      {test_acc:.2f}%")
    print(f"  Time:          {fold_time/60:.1f} min")
    
    fold_results.append({
        'fold': fold_num,
        'test_game': test_game,
        'best_val_acc': best_val_acc,
        'test_acc': test_acc,
        'time_min': fold_time / 60
    })

# Summary
print("\n" + "="*60)
print("2-FOLD VALIDATION COMPLETE")
print("="*60)

fold_df = pd.DataFrame(fold_results)
print(fold_df.to_string(index=False))

mean_test = fold_df['test_acc'].mean()
print(f"\nMean Test Accuracy: {mean_test:.2f}%")
print(f"✓ Model learns effectively across games!" if mean_test > 85 else "⚠ Lower than expected")
print("="*60)

## 5. Full Training on ALL 5 Games (Final Model)

Train a single model on all 5 games combined for production use.

In [ ]:
set_seed(42)

print("="*60)
print("TRAINING ON ALL 5 GAMES COMBINED")
print("="*60)
print(f"Start Time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Load combined dataset
all_manifest = pd.read_csv('dataset_out/all_games_manifest.csv')
train_df = all_manifest[all_manifest['split'] == 'train']
val_df = all_manifest[all_manifest['split'] == 'val']

print(f"\nTrain: {len(train_df):,} squares (80% of all 5 games)")
print(f"Val:   {len(val_df):,} squares (20% of all 5 games)")

# Create datasets
train_dataset = ChessSquareDataset(train_df, transform=train_transform)
val_dataset = ChessSquareDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Initialize model
model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

# Training loop
train_start = time.time()
best_val_acc = 0.0
best_epoch = 0
history = []

print("\nTraining for 8 epochs...")

for epoch in range(EPOCHS):
    epoch_start = time.time()
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    epoch_time = time.time() - epoch_start
    
    history.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'lr': optimizer.param_groups[0]['lr'],
        'time_sec': epoch_time
    })
    
    print(f"  Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"  Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    print(f"  Time:  {epoch_time:.1f}s")
    print(f"  LR:    {optimizer.param_groups[0]['lr']:.6f}")
    
    scheduler.step(val_acc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'train_acc': train_acc,
            'games': 'all_5_games'
        }, 'dataset_out/best_model_all_games.pth')
        print(f"  ✓ Best model saved (Val Acc: {val_acc:.2f}%)")

train_time = time.time() - train_start

# Save history
pd.DataFrame(history).to_csv('dataset_out/all_games_history.csv', index=False)

print("\n" + "="*60)
print("TRAINING ON ALL 5 GAMES COMPLETE")
print("="*60)
print(f"Best Val Acc:    {best_val_acc:.2f}% (epoch {best_epoch})")
print(f"Training Time:   {train_time/60:.1f} minutes")
print(f"Model Saved:     dataset_out/best_model_all_games.pth")
print("\n✓ Final production model ready!")
print("="*60)

## 6. Visualization and Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Load training history
history_df = pd.read_csv('dataset_out/all_games_history.csv')

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training Progress (Loss)
ax1 = axes[0, 0]
ax1.plot(history_df['epoch'], history_df['train_loss'], 'b-o', label='Train Loss', linewidth=2)
ax1.plot(history_df['epoch'], history_df['val_loss'], 'r-o', label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax1.set_title('Training Progress - Loss', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: Training Progress (Accuracy)
ax2 = axes[0, 1]
ax2.plot(history_df['epoch'], history_df['train_acc'], 'b-o', label='Train Acc', linewidth=2)
ax2.plot(history_df['epoch'], history_df['val_acc'], 'r-o', label='Val Acc', linewidth=2)
ax2.axhline(best_val_acc, color='g', linestyle='--', label=f'Best: {best_val_acc:.2f}%')
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
ax2.set_title('Training Progress - Accuracy', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

# Plot 3: 2-Fold Validation Results
ax3 = axes[1, 0]
x = np.arange(2)
width = 0.35
bars1 = ax3.bar(x - width/2, fold_df['best_val_acc'], width, label='Val Acc', 
                color='green', alpha=0.7, edgecolor='black')
bars2 = ax3.bar(x + width/2, fold_df['test_acc'], width, label='Test Acc', 
                color='orange', alpha=0.7, edgecolor='black')
ax3.set_xlabel('Fold', fontsize=12, fontweight='bold')
ax3.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
ax3.set_title('2-Fold Cross-Validation Results', fontsize=14, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(['Fold 1', 'Fold 2'])
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Learning Rate Schedule
ax4 = axes[1, 1]
ax4.plot(history_df['epoch'], history_df['lr'], 'purple', marker='o', linewidth=2)
ax4.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax4.set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
ax4.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
ax4.set_yscale('log')
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('dataset_out/training_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
print("\n2-Fold Validation:")
print(f"  Mean Test Acc: {fold_df['test_acc'].mean():.2f}%")
print(f"  Fold 1: {fold_df.iloc[0]['test_acc']:.2f}% (test on {fold_df.iloc[0]['test_game']})")
print(f"  Fold 2: {fold_df.iloc[1]['test_acc']:.2f}% (test on {fold_df.iloc[1]['test_game']})")

print("\nFinal Model (All 5 Games):")
print(f"  Best Val Acc: {best_val_acc:.2f}%")
print(f"  Best Epoch: {best_epoch}")
print(f"  Training Time: {train_time/60:.1f} min")

print("\n" + "="*60)
print("KEY INSIGHTS:")
print("="*60)
print("✓ Cross-game learning validated (2-fold)")
print("✓ Final model trained on all available data")
print(f"✓ Model achieves {best_val_acc:.2f}% validation accuracy")
print("✓ Ready for production deployment")
print("="*60)

## 7. Download Final Model

In [ ]:
from google.colab import files

print("="*60)
print("DOWNLOADING RESULTS")
print("="*60)

# Download final model
print("\nDownloading best_model_all_games.pth...")
files.download('dataset_out/best_model_all_games.pth')

# Download visualizations
print("Downloading training_results.png...")
files.download('dataset_out/training_results.png')

# Download history
print("Downloading all_games_history.csv...")
files.download('dataset_out/all_games_history.csv')

print("\n✓ Downloads complete!")
print("\nNext steps:")
print("1. Rename best_model_all_games.pth to best_model_fold_1.pth")
print("2. Place in checkpoints/ folder")
print("3. Run: python app.py")
print("4. Access web app at http://localhost:5000")
print("\n" + "="*60)